In [2]:
import numpy as np
import cv2

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

In [3]:
from api.config import EMOTION_LABELS, paths
from keras.models import load_model


face_classifier = cv2.CascadeClassifier(paths.FACE_CLASSIFIER_PATH)  # детектор лица OpenCV
classifier = load_model("/home/vorkov/Workspace/EDA/models/model_pictures_96.h5")  # обученная модель для классификации эмоций
cap = cv2.VideoCapture("/dev/video0")
if not cap.isOpened():
    print("Cannot open camera")
    exit()
    
    
while True:
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # FACE DETECTION
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    faces = face_classifier.detectMultiScale(frame, minNeighbors=12, minSize=(96, 96))
    if len(faces) == 0:
        cv2.imshow("YOLOv8 Tracking", frame)
        continue

    x, y, w, h = faces[0]

    frame_cropped = frame[y:y + h, x:x + w]
    image_face_resized = cv2.resize(frame_cropped, (96, 96), interpolation=cv2.INTER_AREA)
    roi = np.empty((1, 96, 96, 3))
    roi[0] = image_face_resized
    roi = roi / 255

    prediction = classifier.predict(roi)
    emotion = EMOTION_LABELS[np.argmax(prediction)]

    label_emotion = "emotion: {}".format(emotion)
    draw_label(frame, (0, frame.shape[0]-90), label_emotion)

    cv2.imshow("YOLOv8 Tracking", frame)

cap.release()
cv2.destroyAllWindows()

2023-11-01 21:41:37.348405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-01 21:41:37.404233: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-01 21:41:37.404428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

1/1 [==============================] - 0s 20ms/step


2023-11-01 21:41:44.581764: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


1/1 [==============================] - 0s 54ms/step
